# Семантичко Језгро

У овом примеру кода, користићете [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI оквир за креирање основног агента.

Циљ овог примера је да вам покаже кораке које ћемо касније користити у додатним примерима кода приликом имплементације различитих агентичких образаца.


## Увоз потребних Python пакета


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Креирање клијента

У овом примеру користићемо [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за приступ LLM-у.

`ai_model_id` је дефинисан као `gpt-4o-mini`. Пробајте да промените модел на неки други доступан у GitHub Models продавници како бисте видели различите резултате.

Да бисмо користили `Azure Inference SDK`, који се користи за `base_url` за GitHub Models, искористићемо конектор `OpenAIChatCompletion` унутар Semantic Kernel-а. Постоје и други [доступни конектори](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) који омогућавају коришћење Semantic Kernel-а за друге провајдере модела.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Креирање агента

У наставку креирамо агента под називом `TravelAgent`.

За овај пример користимо веома једноставна упутства. Можете променити ова упутства да бисте видели како агент реагује другачије.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Покретање агента

Сада можемо покренути агента дефинисањем нити типа `ChatHistoryAgentThread`. Све потребне системске поруке се прослеђују агенту преко аргумента `messages` у функцији invoke_stream.

Када су ове поруке дефинисане, креирамо `user_inputs`, што представља оно што корисник шаље агенту. У овом случају, поставили смо ову поруку на `Plan me a sunny vacation`.

Слободно промените ову поруку да бисте видели како агент различито реагује.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
